In [178]:
import requests
from bs4 import BeautifulSoup

url = "https://bulldogjob.pl/companies/jobs/s/city,Warszawa,Wrocław,Remote/skills,Python/experienceLevel,medium,junior"

page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")


In [179]:
results = soup.find(id="__next")

In [180]:
page_count = results.find("ul", class_="flex justify-center items-center my-10 pagination-bar")
page_list = list(page_count.text)

In [181]:
page_list

['1', '2']

In [133]:
job_elements = results.find_all("div", class_="py-6 md:py-8 px-8 flex flex-wrap relative bg-white mb-2 rounded-lg shadow")

In [169]:
bulldog_dict = dict(dict())
id_counter = 1
for page in page_list:
    url = f"https://bulldogjob.pl/companies/jobs/s/city,Warszawa,Wrocław,Remote/skills,Python/page,{page}"

    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    for job_element in job_elements:
        company_element = job_element.find("p", class_="text-sm md:text-xxs md:text-center my-2 font-medium text-gray-300")
        job_title_element = job_element.find("h3", class_="text-c28 font-medium mb-3 w-full md:hidden")
        position_element = job_element.find("p", class_="tracking-05 uppercase md:my-4")
        job_types_element = job_element.find("p", class_="font-medium mb-4 md:my-4 flex flex-wrap items-center")
        links = job_element.find("h3", class_="text-c28 font-medium mb-3 w-full md:hidden")
        
        for link in links:
            link_url = link["href"]
        page_job_element = requests.get(link_url)
        soup_page = BeautifulSoup(page_job_element.content, "html.parser")
        date_of_end = soup_page.find("p", class_="text-md xl:text-c22 leading-6")
        if position_element.text.strip() == "junior":
            bulldog_dict[id_counter] = {}
            bulldog_dict[id_counter]["date of end"] = date_of_end.text.strip()
            bulldog_dict[id_counter]["company"] = company_element.text.strip()
            bulldog_dict[id_counter]["title"] = job_title_element.text.strip()
            bulldog_dict[id_counter]["position"] = position_element.text.strip()
            bulldog_dict[id_counter]["link_url"] = link_url



            id_counter += 1


In [176]:
bulldog_dict = dict(dict())
id_counter = 1

url = f"https://bulldogjob.pl/companies/jobs/s/city,Warszawa,Wrocław,Remote/skills,Python/experienceLevel,medium,junior/page,2"

page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

for job_element in job_elements:
    company_element = job_element.find("p", class_="text-sm md:text-xxs md:text-center my-2 font-medium text-gray-300")
    job_title_element = job_element.find("h3", class_="text-c28 font-medium mb-3 w-full md:hidden")
    position_element = job_element.find("p", class_="tracking-05 uppercase md:my-4")
    job_types_element = job_element.find("p", class_="font-medium mb-4 md:my-4 flex flex-wrap items-center")
    links = job_element.find("h3", class_="text-c28 font-medium mb-3 w-full md:hidden")
    
    for link in links:
        link_url = link["href"]
    page_job_element = requests.get(link_url)
    soup_page = BeautifulSoup(page_job_element.content, "html.parser")
    date_of_end = soup_page.find("p", class_="text-md xl:text-c22 leading-6")
    if position_element.text.strip() == "junior":
        bulldog_dict[id_counter] = {}
        bulldog_dict[id_counter]["date of end"] = date_of_end.text.strip()
        bulldog_dict[id_counter]["company"] = company_element.text.strip()
        bulldog_dict[id_counter]["title"] = job_title_element.text.strip()
        bulldog_dict[id_counter]["position"] = position_element.text.strip()
        bulldog_dict[id_counter]["link_url"] = link_url



        id_counter += 1


In [177]:
bulldog_dict

{1: {'date of end': '7.09.2022',
  'company': 'Sii Polska',
  'title': 'Test Automation Engineer',
  'position': 'junior',
  'link_url': 'https://bulldogjob.pl/companies/jobs/118907-test-automation-engineer-gdansk-rzeszow-wroclaw-poznan-krakow-lodz-katowice-czestochowa-bydgoszcz-gliwice-bialystok-szczecin-kielce-torun-sii-polska'}}

In [136]:
import pandas as pd
import numpy as np

In [170]:
df = pd.DataFrame.from_dict(bulldog_dict, orient="index")
#df['date of end'] = pd.to_datetime(df['date of end'], format='%d%m%y')

In [171]:
df

,date of end,company,title,position,link_url
1,7.09.2022,Sii Polska,Test Automation Engineer,junior,https://bulldogjob.pl/companies/jobs/118907-te...
2,7.09.2022,Sii Polska,Test Automation Engineer,junior,https://bulldogjob.pl/companies/jobs/118907-te...
3,7.09.2022,Sii Polska,Test Automation Engineer,junior,https://bulldogjob.pl/companies/jobs/118907-te...


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 1 to 1
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date of end  1 non-null      object
 1   company      1 non-null      object
 2   title        1 non-null      object
 3   position     1 non-null      object
 4   link_url     1 non-null      object
dtypes: object(5)
memory usage: 48.0+ bytes


In [140]:
df['date of end'] = pd.to_datetime(df['date of end'], infer_datetime_format=True)

In [141]:
df

,date of end,company,title,position,link_url
1,2022-07-09,Sii Polska,Test Automation Engineer,junior,https://bulldogjob.pl/companies/jobs/118907-te...


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 1 to 1
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date of end  1 non-null      datetime64[ns]
 1   company      1 non-null      object        
 2   title        1 non-null      object        
 3   position     1 non-null      object        
 4   link_url     1 non-null      object        
dtypes: datetime64[ns](1), object(4)
memory usage: 48.0+ bytes
